In [ ]:
!pip install transformers sentence-transformers langchain
!pip install -U langchain-community
!pip install pypdf
!pip install faiss-cpu
!pip install langchain_huggingface


import pypdf
import os
from google.colab import userdata
from huggingface_hub import InferenceClient
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
import transformers


/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [ ]:


model_name = "microsoft/layoutlm-base-uncased"
tokenizer = transformers.AutoTokenizer.from_pretrained(model_name)
model = transformers.AutoModelForSequenceClassification.from_pretrained(model_name)

embedder = HuggingFaceEmbeddings(model_name="all-mpnet-base-v2")

def extract_text_from_pdf(pdf_file_path):
    """Extracts text from a PDF file using pypdf."""
    with open(pdf_file_path, 'rb') as f:
        pdf_reader = pypdf.PdfReader(f)
        text = ""
        for page in pdf_reader.pages:
            text += page.extract_text()
    return text

def create_faiss_index(text_data, embedder):
    """Creates a FAISS index for text data."""
    chunks = text_data.split("\n")

    vectorstore = FAISS.from_texts(chunks, embedder)
    return vectorstore

def handle_user_input(user_question, pdf_file_path):
    """Handles user input, searches for relevant text, and generates a response."""

    pdf_text = extract_text_from_pdf(pdf_file_path)

    global vectorstore
    try:
        vectorstore
    except NameError:
        vectorstore = create_faiss_index(pdf_text, embedder)

    retriever = vectorstore.as_retriever()
    relevant_text = retriever.get_relevant_documents(user_question)

    response = generate_response_from_inference_api(user_question, relevant_text)

    return response

def generate_response_from_inference_api(user_question, relevant_text):
    """Sends a request to the Hugging Face Inference API."""
    sec_key = userdata.get("HUGGINGFACEHUB_API_TOKEN")
    os.environ["HUGGINGFACEHUB_API_TOKEN"] = sec_key

    url = "https://api-inference.huggingface.co/models/google/flan-t5-large"
    headers = {"Authorization": f"Bearer {os.environ['HUGGINGFACEHUB_API_TOKEN']}"}

    prompt = f"""
    The PDF document says: {relevant_text}
    Question: {user_question}
    Answer:
    """
    data = {"inputs": prompt}
    response = requests.post(url, headers=headers, json=data)
    if response.status_code == 200:
        return response.json()[0]["generated_text"]
    else:
        return f"Error: {response.status_code}"

pdf_file_path = "budget_speech.pdf"
user_question = "what is the Vision for Amrit Kaal?"
response = handle_user_input(user_question, pdf_file_path)
print(response)

Some weights of LayoutLMForSequenceClassification were not initialized from the model checkpoint at microsoft/layoutlm-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


pursuit of ‘ Viksit Bharat’. In line with the strategy set out


In [ ]:
pdf_file_path = "budget_speech.pdf"
user_question =  "How much the agriculture target will be increased to "
response = handle_user_input(user_question, pdf_file_path)
print(response)

1 crore farmers across the country will be
